# 1 - Imports and defining functions

In [1]:
import matplotlib.pyplot as plt
import open3d as o3d
import numpy as np
import torch
import scipy.optimize as so
import meshplot as mp

import pyFM.spectral as spectral
from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping
from project.cfunctional import CoupledFunctionalMapping
from project.trimesh import TriMeshPly
import project.utils as file_utils
import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

#def get_pts(infile):
#    data = np.loadtxt(infile, delimiter=',')
#    return data[12:,0], data[12:,1], data[12:,2] #returns X,Y,Z points skipping the first 12 lines
        
def plot_ply(infile):
        
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x,y,z = get_pts(infile)
    ax.scatter(x, y, z, c='r', marker='o')
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    plt.show()
 


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 2 - Try with the FAUST dataset

**Loading meshes from FAUST dataset***

Create the TriMesh class can be created from a path. With the load_ply we use two meshes from the FAUST dataset 

In [2]:
FAUST_path_1 = str("../data/MPI-FAUST_dev/MPI-FAUST/training/registrations/tr_reg_000.ply")
FAUST_path_2 = str("../data/MPI-FAUST_dev/MPI-FAUST/training/registrations/tr_reg_001.ply")


mesh1 = TriMeshPly(FAUST_path_1)
mesh2 = TriMeshPly(FAUST_path_2)


print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

double_plot(mesh1,mesh2)

Mesh 1 : 6890 vertices, 13776 faces
Mesh 2 : 6890 vertices, 13776 faces


C:\Users\Admin\anaconda3\envs\casestud\lib\site-packages\jupyter_client\session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


**Computing descriptors for the functional map**

In [3]:
process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
#    'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5], #loading 5 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
}

model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);



Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 2.06 s
Computing 200 eigenvectors
	Done in 1.89 s

Computing descriptors
	Normalizing descriptors

	20 out of 100 possible descriptors kept


**Fitting the model for the functional map**

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\Ss}{\mathcal{S}}$
$\newcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\newcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

Compute the functional map $\*C:\Ss_1\to\Ss_2$:

Optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1}} w_{descr}\|\*C\*A - \*B\|^2 + w_{lap}\|\*C\Delta_1 - \Delta_2\*C\|^2 + w_{\text{d- comm}}\sum_i \|\*C\Gamma_1^i - \Gamma_2^i\*C\|^2 + w_{\text{orient}}\sum_i \|\*C\Lambda_1^i - \Lambda_2^i\*C\|^2
\end{equation}

with $\Gamma_1^i$ and $\Gamma_2^i$ associated to the $i$-th descriptors, $\Lambda_1^i$ and $\Lambda_2^i$ associated to the $i$-th descriptors

In [4]:
fit_params = {
    'w_descr': 1e0,
    'w_lap': 1e-2,
    'w_dcomm': 1e-1,
    'w_orient': 0
}



model.fit(**fit_params, verbose=True)

Computing commutativity operators
	Scaling LBO commutativity weight by 7.0e-08

Optimization :
	35 Ev on source - 35 Ev on Target
	Using 20 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 181, nit : 179, warnflag : 0
	Done in 0.35 seconds


**Visualizing the associated point to point map**

In [5]:
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

C:\Users\Admin\anaconda3\envs\casestud\lib\site-packages\jupyter_client\session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [6]:
gt_1 = np.loadtxt("../data/MPI-FAUST_dev/MPI-FAUST/training/ground_truth_vertices/tr_gt_000.txt")
gt_2 = np.loadtxt("../data/MPI-FAUST_dev/MPI-FAUST/training/ground_truth_vertices/tr_gt_001.txt")
print()

***Computing the descriptors for the coupled functional maps***

In [7]:
process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
#    'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5], #loading 5 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
}

model2 = CoupledFunctionalMapping(mesh1,mesh2)
model2.preprocess(**process_params,verbose=True);


Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 1.96 s
Computing 200 eigenvectors
	Done in 1.85 s

Computing descriptors
	Normalizing descriptors

	20 out of 100 possible descriptors kept


**Fitting the model for the coupled functional maps**

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\Ss}{\mathcal{S}}$
$\newcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\newcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

Compute the coupled functional maps $\*C_1:\Ss_1\to\Ss_2$ and $\*C_2:\Ss_2\to\Ss_1$:

Optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1},\*C\in\RR^{k_1\times k_2}} \|\*C_1\*A - \*B\|^2 + \|\*A-\*C_2 \*B\|^2 + \mu_{coup}\|\*C_1\*C_2-\*I\|^2 + \mu_{LB}\|\*C_1\Delta_1 - \Delta_2\*C_1\|^2 + \mu_{LB}\|\*C_2^{T}\Delta_1 - \Delta_2\*C_2^{T}\|^2 
\end{equation}



In [8]:
fit_params = {
    'mu_cons': 1e-1,
    'mu_LB': 1e3
}

model2.fit(**fit_params, verbose=True)

         Current function value: 0.050927
         Iterations: 150
         Function evaluations: 350
         Gradient evaluations: 338


**Visualizing the associated point to point map**

In [9]:
cp2p = model2.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[cp2p]
double_plot(mesh1,mesh2,cmap1,cmap2)

C:\Users\Admin\anaconda3\envs\casestud\lib\site-packages\jupyter_client\session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


# 3 - try with the cat-lion datas

**Loading meshes of the cat and the Lion***

In [10]:
cat_mesh = TriMesh('data/cat-00.off')
lion_mesh = TriMesh('data/lion-00.off')
print(f'Mesh 1 : {cat_mesh.n_vertices:4d} vertices, {cat_mesh.n_faces:5d} faces\n'
      f'Mesh 2 : {lion_mesh.n_vertices:4d} vertices, {lion_mesh.n_faces:5d} faces')

Mesh 1 : 7207 vertices, 14410 faces
Mesh 2 : 5000 vertices,  9996 faces


**Computing the descriptors**

In [11]:
process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
    'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5],  # loading 5 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
}

model1_CL = FunctionalMapping(cat_mesh,lion_mesh)
model1_CL.preprocess(**process_params,verbose=True);


Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 2.15 s
Computing 200 eigenvectors
	Done in 1.04 s

Computing descriptors
	Normalizing descriptors

	120 out of 600 possible descriptors kept


**Fitting the model for the functional map**

In [12]:
fit_params = {
    'w_descr': 1e0,
    'w_lap': 1e3,
    'w_dcomm': 1e-2,
    'w_orient': 0
}



model1_CL.fit(**fit_params, verbose=True)

Computing commutativity operators
	Scaling LBO commutativity weight by 3.9e-09

Optimization :
	35 Ev on source - 35 Ev on Target
	Using 120 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-02
		Laplacian commutativity :1.0e+03
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 43, nit : 38, warnflag : 0
	Done in 0.29 seconds


**Visualizing the associated point to point map for the functional map**

In [13]:
p2p_CL = model1_CL.get_p2p(n_jobs=1)
cmap1 = visu(cat_mesh.vertlist); cmap2 = cmap1[p2p_CL]
double_plot(cat_mesh,lion_mesh,cmap1,cmap2)

C:\Users\Admin\anaconda3\envs\casestud\lib\site-packages\jupyter_client\session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


**Computing the descriptors**

In [14]:
process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
    'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5], #loading 5 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
}



model2_CL = CoupledFunctionalMapping(cat_mesh,lion_mesh)
model2_CL.preprocess(**process_params,verbose=True);


Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 2.15 s
Computing 200 eigenvectors
	Done in 1.01 s

Computing descriptors
	Normalizing descriptors

	120 out of 600 possible descriptors kept


**Fitting the model for the coupled functional map**

In [15]:
fit_params = {
    'mu_cons': 1e-1,
    'mu_LB': 1e3,
}

model2_CL.fit(**fit_params, verbose=True)
#print(model2.C1)

         Current function value: 11.867128
         Iterations: 20
         Function evaluations: 90
         Gradient evaluations: 79


**Visualizing the associated point to point map for the coupled functional map**

In [16]:
cp2p_CL = model2_CL.get_p2p(n_jobs=1)
cmap1_C = visu(cat_mesh.vertlist); cmap2_C = cmap1_C[cp2p_CL]
double_plot(cat_mesh,lion_mesh,cmap1_C,cmap2_C)

C:\Users\Admin\anaconda3\envs\casestud\lib\site-packages\jupyter_client\session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


# Evaluating Results

In [17]:
import pyFM.eval

In [18]:
A_geod = cat_mesh.get_geodesic(verbose=True)

  0%|          | 0/7207 [00:00<?, ?it/s]

In [19]:
# Load an approximate ground truth map
gt_p2p = np.loadtxt('data/lion2cat',dtype=int)

acc_FM = pyFM.eval.accuracy(p2p_CL, gt_p2p, A_geod, sqrt_area=cat_mesh.sqrtarea)

acc_CFM = pyFM.eval.accuracy(cp2p_CL, gt_p2p, A_geod, sqrt_area=cat_mesh.sqrtarea)


print(f'Accuracy results:\n'
      f'\tFM : {1e3*acc_FM:.2f}\n'
      f'\tCFM : {1e3*acc_CFM:.2f}\n')

Accuracy results:
	FM : 40.86
	CFM : 56.86

